In [ ]:
!pip install datasets
!pip install -U transformers
!pip install ace_tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, default_data_collator
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm
import pandas as pd
from peft import get_peft_model, LoraConfig, PromptTuningConfig


# Базовая модель, загрузка датасета, токенизация


In [ ]:
# Загружаем объединённый датасет (train + validation + test)
ds = load_dataset('dair-ai/emotion', 'split')

model_checkpoint = 'google-bert/bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Токенизация
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

# Токенизируем все части датасета
ds_encoded = ds.map(tokenize, batched=True)

# Устанавливаем формат данных для работы с PyTorch
ds_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Теперь можно использовать ds_encoded['train'], ds_encoded['validation'], ds_encoded['test'] для тренировки модели

# Токенизируем все части
ds_encoded = ds.map(tokenize, batched=True)

# Преобразуем названия меток в id
label2id = {label: i for i, label in enumerate(ds_encoded['train'].features['label'].names)}
id2label = {i: label for label, i in label2id.items()}
num_labels = len(label2id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

# Функция для расчета метрики

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

## Full-finetuning model

Full Finetuning (1 балл)

    Fine-tune всей модели BERT + Classification Head

    Использовать Trainer или PyTorch напрямую

    Учим все веса, включая BERT

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Обучение, метрики

До начала экспериментов, обучите модель в режиме full finetuning и зафиксируйте:

    Accuracy, F1-score

    Время обучения

    Кол-во обучаемых параметров

    Использование памяти GPU

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_encoded['train'],
    eval_dataset=ds_encoded['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model('./model_full_finetuning')

<ipython-input-5-1609bb8e8b1d>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.719300
20,1.621700
30,1.583700
40,1.455400
50,1.438900
60,1.318900
70,1.309300
80,1.266300
90,1.174400
100,1.090100


In [ ]:
# Оценка на тесте
results1 = trainer.evaluate(ds_encoded['test'])
print('Test results:', results1)

Test results: {'eval_loss': 0.16732938587665558, 'eval_accuracy': 0.928, 'eval_f1': 0.927267027187444, 'eval_runtime': 14.2114, 'eval_samples_per_second': 140.732, 'eval_steps_per_second': 4.433, 'epoch': 3.0}


In [ ]:
# Метрики ресурсоёмкости
# Обучаемые параметры
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Trainable parameters:', trainable_params)

# Используемая память CUDA
print('CUDA memory allocated (MB):', torch.cuda.memory_allocated() / 1024**2)

Trainable parameters: 109486854
CUDA memory allocated (MB): 1314.47705078125


In [ ]:
acc_full = results1['eval_accuracy']
f1_full = results1['eval_f1']
params_full = trainable_params
mem_full = torch.cuda.memory_allocated() / 1024**2

# Linear Probing (2 балла)

    Замораживаем BERT

    Обучаем только кастомную голову

In [ ]:
# Загружаем модель BERT для классификации
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

'''
На выходе модели BERT, после обработки входных данных, мы получаем скрытые состояния для каждого токена в предложении.
используется только скрытое состояние для [CLS] токена, которое является эмбеддингом для всего предложения.
Полносвязный слой получает эмбеддинг [CLS] и преобразует его в выходные значения, соответствующие количеству классов - это и есть классификационная голова в нашем случае.
Остальные слои заморожены, т.к. модель уже предобучена на большом корпусе текстов и понимает общие закономерности языка. Замораживая слои, мы как раз обеспечиваем возможность использовать уже обученные веса.
С помощью обучаемой классификационной головы модель адаптируется под конкретную задачу, например, для классификации эмоций в тексте.
num_labels = 6 , т.к. 6 эмоций
'''

# Замораживаем все слои модели, кроме классификационной головы
for param in model.base_model.parameters():
    param.requires_grad = False

# Выводим информацию о том, что только голова модели будет обучаться
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Training {name}")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training classifier.weight
Training classifier.bias


In [ ]:
# Настройки обучения
training_args = TrainingArguments(
    output_dir='./results',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    report_to='none'
)

# Создаём объект Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_encoded['train'],
    eval_dataset=ds_encoded['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Тренировка модели
trainer.train()

<ipython-input-11-a700f3a77987>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Сохранение модели
trainer.save_model('./model_linear_probing')

# Оценка на тестовой выборке
results2 = trainer.evaluate(ds_encoded['test'])
print(results2)

{'eval_loss': 1.54506516456604, 'eval_accuracy': 0.3635, 'eval_f1': 0.2551102916404224, 'eval_runtime': 13.4696, 'eval_samples_per_second': 148.482, 'eval_steps_per_second': 4.677, 'epoch': 3.0}


In [ ]:
acc_linear = results2['eval_accuracy']
f1_linear = results2['eval_f1']
params_linear = sum(p.numel() for p in model.parameters() if p.requires_grad)
mem_linear = torch.cuda.memory_allocated() / 1024**2

# Prompt-tuning

In [ ]:
# Сбрасываем формат, чтобы default_data_collator получил списки:
ds_encoded.reset_format()

train_loader = DataLoader(
    ds_encoded['train'],
    batch_size=16,
    shuffle=True,
    collate_fn=default_data_collator
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

base_model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=6
).to(device)

prompt_config = PromptTuningConfig(
    task_type='SEQ_CLS',   # задаём задачу как классификацию последовательностей
    num_virtual_tokens=25,  # например, 5 виртуальных токенов
    #tokenizer_name='bert-base-uncased'  # имя токенизатора для соответствия с моделью
)

# Оборачиваем базовую модель в PEFT-модель
prompt_model = get_peft_model(base_model, prompt_config)
prompt_model.print_trainable_parameters()  # Проверяем, сколько параметров обучается

# Определяем оптимизатор
optimizer = torch.optim.Adam(prompt_model.parameters(), lr=5e-5)

# Цикл обучения с tqdm
all_labels = []
all_preds = []

# Цикл обучения с tqdm
for epoch in range(1, 4):
    prompt_model.train()
    total_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch {epoch}", leave=False)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch.get('labels', batch.get('label'))

        # Получаем предсказания
        outputs = prompt_model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=labels
        )
        loss = outputs.loss
        logits = outputs.logits

        # Собираем метки и предсказания
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(torch.argmax(logits, dim=-1).cpu().numpy())

        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch} — avg loss: {avg_loss:.4f}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 23,814 || all params: 109,510,668 || trainable%: 0.0217


Epoch 1:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1 — avg loss: 1.5923


Epoch 2:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
if len(all_labels) > 0 and len(all_preds) > 0:
    accuracy_lora = accuracy_score(all_labels, all_preds)
    f1_score_lora = f1_score(all_labels, all_preds, average='weighted')
else:
    accuracy_lora = f1_score_lora = None
    print("Предсказания или метки не были собраны.")

# Параметры и память
params_prompt_lora = sum(p.numel() for p in prompt_model.parameters() if p.requires_grad)
mem_prompt_lora = torch.cuda.memory_allocated() / 1024**2 if torch.cuda.is_available() else 0

# Вывод результатов
print('Accuracy:', accuracy_lora)
print('F1-score:', f1_score_lora)
print(f'Trainable params: {params_prompt_lora} || All params: {sum(p.numel() for p in prompt_model.parameters())}')
print(f'GPU Memory (MB): {mem_prompt_lora:.2f}')

Accuracy: 0.35422916666666665
F1-score: 0.2632528717221229
Trainable params: 8454 || All params: 109495308
GPU Memory (MB): 436.39


In [ ]:
results3 = trainer.evaluate(ds_encoded['test'])

acc_linear = results3['eval_accuracy']
f1_linear = results3['eval_f1']
params_prompt = sum(p.numel() for p in model.parameters() if p.requires_grad)
mem_prompt = torch.cuda.memory_allocated() / 1024**2

print('Accuracy:', acc_linear)
print('F1-score:', f1_linear)

# PEFT с использованием LoRA

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Базовая модель + классификатор
base_model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=6
).to(device)

# Конфигурация LoRA
# r=8 выбран как компромисс между адаптивностью и невысокой нагрузкой на память.
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['query', 'value'],
    lora_dropout=0.1,
    bias='none',
    task_type='SEQ_CLS'
)

# Оборачиваем базовую модель в PEFT‑LoRA
lora_model = get_peft_model(base_model, lora_config)
lora_model.print_trainable_parameters()  # сколько параметров обучается

# DataLoader (работаем с «сырыми» примерами, сбрасываем формат, если нужно)
ds_encoded.reset_format()
train_loader = DataLoader(
    ds_encoded['train'],
    batch_size=16,
    shuffle=True,
    collate_fn=default_data_collator
)
optimizer = torch.optim.Adam(lora_model.parameters(), lr=3e-4)
# Цикл обучения
for epoch in range(1, 4):
    lora_model.train()
    total_loss = 0.0
    loop = tqdm(train_loader, desc=f"LoRA Epoch {epoch}", leave=False)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch.get('labels', batch.get('label'))

        optimizer.zero_grad()
        outputs = lora_model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=labels
        )
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f'LoRA Epoch {epoch} — avg loss: {avg_loss:.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 299,526 || all params: 109,786,380 || trainable%: 0.2728


LoRA Epoch 1:   0%|          | 0/1000 [00:00<?, ?it/s]

LoRA Epoch 1 — avg loss: 0.7486


LoRA Epoch 2:   0%|          | 0/1000 [00:00<?, ?it/s]

LoRA Epoch 2 — avg loss: 0.2489


LoRA Epoch 3:   0%|          | 0/1000 [00:00<?, ?it/s]

LoRA Epoch 3 — avg loss: 0.1813


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
lora_model.eval()
test_loader = DataLoader(
    ds_encoded['test'],
    batch_size=16,
    shuffle=False,
    collate_fn=default_data_collator
)

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluating LoRA'):
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch.get('labels', batch.get('label'))

        outputs = lora_model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask']
        )
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Считаем метрики
accuracy_lora = accuracy_score(all_labels, all_preds)
f1_score_lora = f1_score(all_labels, all_preds, average='weighted')

# Параметры и память
params_lora = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
mem_lora = torch.cuda.memory_allocated() / 1024**2

print(f'Accuracy: {accuracy_lora:.4f}')
print(f'F1-score: {f1_score_lora:.4f}')
print(f'Trainable parameters: {params_lora}')
print(f'GPU memory used: {mem_lora:.2f} MB')

Evaluating LoRA:   0%|          | 0/125 [00:00<?, ?it/s]

Accuracy: 0.9280
F1-score: 0.9270
Trainable parameters: 299526
GPU memory used: 1730.19 MB


# Итог

In [ ]:
results = pd.DataFrame({
    'Method': ['Full Finetuning', 'Linear Probing', 'Prompt Tuning', 'LoRA (r=8)'],
    'Accuracy': [acc_full, acc_linear, acc_linear, accuracy_lora],
    'F1-score': [f1_full, f1_linear, f1_linear, f1_score_lora],
    'Trainable Params': [params_full, params_linear, params_prompt, params_lora],
    'Time for learning': ['16m30s', '6m00s', '19m35s', '12m00s'],
    'GPU Memory (MB)': [mem_full, mem_linear, mem_prompt, mem_lora]
})
results

,Method,Accuracy,F1-score,Trainable Params,Time for learning,GPU Memory (MB)
0,Full Finetuning,0.9280,0.927267,109486854,16m30s,1314.477051
1,Linear Probing,0.3635,0.255110,4614,6m00s,458.283691
2,Prompt Tuning,0.3635,0.255110,109490694,19m35s,2144.232422
3,LoRA (r=8),0.9280,0.926965,299526,12m00s,1730.188965


# Выводы

Выполнено обучение моделей на одинаковом датасете:

1. Full Finetuning

    Дообучили все веса модели bert-base-uncased + классификационная голова.

    Использовали Trainer из 🤗 Transformers.

    Собрали метрики: accuracy, F1, обучаемые параметры, GPU usage, время.

2. Linear Probing

    Заморозили все слои BERT, обучалась только классификационная голова.

    Минимум обучаемых параметров, быстрая и экономная тренировка.

    Использовали тот же Trainer.

3. Prompt Tuning

    Добавили обучаемые prompt-эмбеддинги перед входом в BERT.

    Обернули модель в кастомный PyTorch-класс.

    Использовали DataLoader, tqdm и ручной тренировочный цикл.

4. LoRA (Low-Rank Adaptation)

    Адаптировали только малую часть модели через PEFT (с peft и LoraConfig).

    На практике — отличный компромисс: быстрее и экономичнее full-finetuning.

    Также использовали кастомный цикл


Сравнительные результаты отражены в таблице "итоги".
Обучение проводилось всего на 3 эпохах по причине ограниченности вычислительных ресурсов.

Full Finetuning показал лучшую метрику наравне с LORA, но требует:

    Огромных ресурсов: памяти, времени и обучаемых параметров (109 млн.)

LoRA показывает отличные результаты:

    Такая же точность, как при полном дообучении.

    В 300+ раз меньше обучаемых параметров, что особенно важно для работы с ограниченными ресурсами. Время обучения на 25% короче, чем full.


Linear Probing предсказуемо слаб:

    Без адаптации эмбеддингов модель не может хорошо обобщать. Число параметров при этом всего 4000

Prompt Tuning требует доработки:

    Результаты не лучше, чем у Linear Probing, а ресурсов требует почти как Full Finetuning.
